In [ ]:
import findspark 
findspark.init() 
from pyspark.sql import SparkSession

spark=SparkSession.builder.appName("Test")\
        .config("spark.sql.warehouse.dir", "../../data/spark-warehouse")\
        .config('spark.driver.extraJavaOptions',f'-Dderby.system.home=../../data/catalog')\
        .config("spark.io.encryption.enabled",True)\
        .config('spark.acls.enable',True)\
        .enableHiveSupport().getOrCreate()
spark

## restart kernel if testing needed for new  changes of  spark-warehouse 

Tests to check if data & metadata stored correctly in storage

In [1]:
## list of all tables stored in spark-warehouse 
spark.catalog.listTables()

In [ ]:
## restaurants data
spark.sql('select * from restaurant limit 20').show()


In [ ]:
## consumers data
spark.sql('select * from consumer limit 20').show()

In [ ]:
## orders data

spark.sql('select * from order limit 20').show()

In [ ]:
## test order response query
spark.sql("""
        select  delivery_address_state,delivery_address_city, date(order_created_at), count(*) as num_orders
        from order 
        group by delivery_address_state,delivery_address_city,date(order_created_at) 
        order by delivery_address_state,delivery_address_city,date(order_created_at)
        """).show(200,False)

In [ ]:
## test customers top 10 restaurants


spark.sql("""
        with customer_orders as (
            select c.customer_id,c.customer_name,o.merchant_id
            from consumer c
            left join order o 
            on c.customer_id = o.customer_id
            ),
        customer_total_restaurants as (
            select co.customer_id,co.customer_name, r.id restaurant_id, count(*) total_orders
            from customer_orders co
            left join  restaurant r
            on co.merchant_id=r.id
            group by co.customer_id,co.customer_name, r.id
            ),
        restaurant_rank as (
            select *, 
            row_number() over (partition by customer_id  order by total_orders desc) rank 
            from customer_total_restaurants
        )

        select * 
        from restaurant_rank
        where rank<=10

        """ ).show(200,False)
